# Multi-Class Classification Deep Learning Model for [Sample Project] Using Keras Version 1
### David Lowe
### October 18, 2019
Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: [Sample Paragraph - The purpose of this project is to construct a predictive model using various machine learning algorithms and to document the end-to-end steps using a template. The Iris Diabetes dataset is a multi-class classification situation where we are trying to predict one of several (more than two) possible outcomes.]

INTRODUCTION: [Sample Paragraph - This is perhaps the best known database to be found in the pattern recognition literature. Fisher's paper is a classic in the field and is referenced frequently to this day. (See Duda & Hart, for example.) The data set contains three classes of 50 instances each, where each class refers to a type of iris plant. One class is linearly separable from the other two; the latter are NOT linearly separable from each other.]

ANALYSIS: [Sample Paragraph - The baseline performance of the model achieved an average accuracy score of 96.67%, which was consistent with other classical machine learning algorithms.]

CONCLUSION: [Sample Paragraph - For this dataset, the model built using Keras and TensorFlow achieved a satisfactory result and should be considered for future modeling activities]

Dataset Used: [Iris Data Set]

Dataset ML Model: Multi-class classification with numerical attributes

Dataset Reference: [https://archive.ics.uci.edu/ml/machine-learning-databases/iris/]

One potential source of performance benchmarks: [https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/]

Any deep-learning modeling project genrally can be broken down into about six major tasks:
0. Prepare Environment
1. Load Data
2. Define Model
3. Fit and Evaluate Model
4. Optimize Model
5. Finalize Model

# Section 0. Prepare Environment

In [65]:
# Create the random seed numbers for reproducible results
seedNum = 88

In [66]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import pandas as pd
import os
import smtplib
from datetime import datetime
from email.message import EmailMessage
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [67]:
# Configure a new global `tensorflow` session
import keras as K
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import tensorflow as tf
tf.set_random_seed(seedNum)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.backend.set_session(sess)

In [68]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set the flag for splitting the dataset
splitDataset = True
splitPercentage = 0.25

# Set various default Keras modeling parameters
model_kernel_init = K.initializers.glorot_uniform(seed=seedNum)
model_loss = 'categorical_crossentropy'
model_optimizer = 'adam'
model_metrics = ['accuracy']
model_epochs = 200
model_batches = 5

# Set the number of folds for cross validation
folds = 10

In [69]:
# Set up the email notification function
def email_notify(msg_text):
    sender = os.environ.get('MAIL_SENDER')
    receiver = os.environ.get('MAIL_RECEIVER')
    gateway = os.environ.get('SMTP_GATEWAY')
    smtpuser = os.environ.get('SMTP_USERNAME')
    password = os.environ.get('SMTP_PASSWORD')
    if sender==None or receiver==None or gateway==None or smtpuser==None or password==None:
        sys.exit("Incomplete email setup info. Script Processing Aborted!!!")
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from Keras Binary Classification Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP(gateway, 587)
    server.starttls()
    server.login(smtpuser, password)
    server.send_message(msg)
    server.quit()

In [70]:
if (notifyStatus): email_notify("Phase 0 Prepare Environment completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 1. Load Data

In [71]:
if (notifyStatus): email_notify("Phase 1 Load Data has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [72]:
# Load the dataset
df_original = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None)
dataset = df_original.values
print(dataset)

[[5.1 3.5 1.4 0.2 'Iris-setosa']
 [4.9 3.0 1.4 0.2 'Iris-setosa']
 [4.7 3.2 1.3 0.2 'Iris-setosa']
 [4.6 3.1 1.5 0.2 'Iris-setosa']
 [5.0 3.6 1.4 0.2 'Iris-setosa']
 [5.4 3.9 1.7 0.4 'Iris-setosa']
 [4.6 3.4 1.4 0.3 'Iris-setosa']
 [5.0 3.4 1.5 0.2 'Iris-setosa']
 [4.4 2.9 1.4 0.2 'Iris-setosa']
 [4.9 3.1 1.5 0.1 'Iris-setosa']
 [5.4 3.7 1.5 0.2 'Iris-setosa']
 [4.8 3.4 1.6 0.2 'Iris-setosa']
 [4.8 3.0 1.4 0.1 'Iris-setosa']
 [4.3 3.0 1.1 0.1 'Iris-setosa']
 [5.8 4.0 1.2 0.2 'Iris-setosa']
 [5.7 4.4 1.5 0.4 'Iris-setosa']
 [5.4 3.9 1.3 0.4 'Iris-setosa']
 [5.1 3.5 1.4 0.3 'Iris-setosa']
 [5.7 3.8 1.7 0.3 'Iris-setosa']
 [5.1 3.8 1.5 0.3 'Iris-setosa']
 [5.4 3.4 1.7 0.2 'Iris-setosa']
 [5.1 3.7 1.5 0.4 'Iris-setosa']
 [4.6 3.6 1.0 0.2 'Iris-setosa']
 [5.1 3.3 1.7 0.5 'Iris-setosa']
 [4.8 3.4 1.9 0.2 'Iris-setosa']
 [5.0 3.0 1.6 0.2 'Iris-setosa']
 [5.0 3.4 1.6 0.4 'Iris-setosa']
 [5.2 3.5 1.5 0.2 'Iris-setosa']
 [5.2 3.4 1.4 0.2 'Iris-setosa']
 [4.7 3.2 1.6 0.2 'Iris-setosa']
 [4.8 3.1 

In [73]:
# Split the original dataset into input (X) and output (y) variables
X_original = dataset[:,0:4].astype(float)
y_original = dataset[:,4]
print('Shape of X_original:', X_original.shape, '| Shape of y_original:', y_original.shape)

Shape of X_original: (150, 4) | Shape of y_original: (150,)


In [74]:
# Encode class values as integers and perform one-hot-encoding
encoder = LabelEncoder()
encoder.fit(y_original)
encoded_y = encoder.transform(y_original)
y_original_dummy = np_utils.to_categorical(encoded_y)
print(y_original_dummy)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

In [75]:
# Split the data further into training and test datasets
if (splitDataset):
    X_train, X_test, y_train, y_test = train_test_split(X_original, y_original_dummy, test_size=splitPercentage, random_state=seedNum)
else:
    X_train, y_train = X_original, y_original_dummy
    X_test, y_test = X_original, y_original_dummy
print('Shape of X_train:', X_train.shape, '| Shape of y_train:', y_train.shape)
print('Shape of X_test:', X_test.shape, '| Shape of y_test:', y_test.shape)

Shape of X_train: (112, 4) | Shape of y_train: (112, 3)
Shape of X_test: (38, 4) | Shape of y_test: (38, 3)


In [76]:
if (notifyStatus): email_notify("Phase 1 Load Data completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 2. Define Model

In [77]:
if (notifyStatus): email_notify("Phase 2 Define Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [78]:
# Define the Keras model required for KerasClassifier
def create_cv_model():
    model = K.models.Sequential()
    model.add(Dense(8, input_dim=4, kernel_initializer=model_kernel_init, activation='relu'))
    model.add(Dense(3, kernel_initializer=model_kernel_init, activation='softmax'))
    model.compile(loss=model_loss, optimizer=model_optimizer, metrics=model_metrics)
    return model

In [79]:
# Initialize the Keras model
cv_model = KerasClassifier(build_fn=create_cv_model, epochs=model_epochs, batch_size=model_batches, verbose=0)

In [80]:
if (notifyStatus): email_notify("Phase 2 Define Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 3. Fit and Evaluate Model

In [81]:
if (notifyStatus): email_notify("Phase 3 Fit and Evaluate Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [82]:
# Fit and evaluate the Keras model using 10-fold cross validation
kfold = KFold(n_splits=folds, shuffle=True, random_state=seedNum)
results = cross_val_score(cv_model, X_train, y_train, cv=kfold)
print('Generating results using the metrics of', model_metrics)
print('All cross-Validate results:', results)
print('Average cross-Validate results:', results.mean())

Generating results using the metrics of ['accuracy']
All cross-Validate results: [1.         1.         0.90909091 1.         1.         1.
 1.         1.         1.         0.90909091]
Average cross-Validate results: 0.9818181829019025


In [83]:
if (notifyStatus): email_notify("Phase 3 Fit and Evaluate Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 4. Optimize Model

In [84]:
if (notifyStatus): email_notify("Phase 4 Optimize Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [85]:
# Define the Keras model required for KerasClassifier
def create_grid_model(optimizer, kernel_init):
    model = K.models.Sequential()
    model.add(Dense(8, input_dim=4, kernel_initializer=kernel_init, activation='relu'))
    model.add(Dense(3, kernel_initializer=kernel_init, activation='softmax'))
    model.compile(loss=model_loss, optimizer=optimizer, metrics=model_metrics)
    return model

In [86]:
# create model
grid_model = KerasClassifier(build_fn=create_grid_model, verbose=0)

# Perform grid search using different epochs, batch sizes, and optimizers
optimizer_grid = ['rmsprop', 'adam']
init_grid = ['Constant', 'RandomNormal', 'RandomUniform']
epoch_grid = [100, 150, 200]
batch_grid = [5, 10, 20]
param_grid = dict(optimizer=optimizer_grid, kernel_init=init_grid, epochs=epoch_grid, batch_size=batch_grid)
grid = GridSearchCV(estimator=grid_model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.973214 using {'batch_size': 5, 'epochs': 150, 'kernel_init': 'RandomNormal', 'optimizer': 'adam'}
0.276786 (0.067255) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'Constant', 'optimizer': 'rmsprop'}
0.276786 (0.067255) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'Constant', 'optimizer': 'adam'}
0.937500 (0.102346) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'RandomNormal', 'optimizer': 'rmsprop'}
0.964286 (0.051226) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'RandomNormal', 'optimizer': 'adam'}
0.875000 (0.164604) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'RandomUniform', 'optimizer': 'rmsprop'}
0.964286 (0.051226) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'RandomUniform', 'optimizer': 'adam'}
0.276786 (0.067255) with: {'batch_size': 5, 'epochs': 150, 'kernel_init': 'Constant', 'optimizer': 'rmsprop'}
0.276786 (0.067255) with: {'batch_size': 5, 'epochs': 150, 'kernel_init': 'Constant', 'optimizer': 'adam'}
0.964286

In [87]:
best_optimizer = 'adam'
best_kernel_init = 'RandomNormal'
best_epoch = 150
best_batch = 5

In [88]:
# Create the final model for evaluating the test dataset
model = create_grid_model(best_optimizer, best_kernel_init)
model.fit(X_train, y_train, epochs=best_epoch, batch_size=best_batch, verbose=1)

Epoch 1/150
112/112 [==============================] - 1s 10ms/step - loss: 1.1075 - acc: 0.0446
Epoch 2/150
112/112 [==============================] - 0s 534us/step - loss: 1.0949 - acc: 0.3125
Epoch 3/150
112/112 [==============================] - 0s 528us/step - loss: 1.0872 - acc: 0.3125
Epoch 4/150
112/112 [==============================] - 0s 515us/step - loss: 1.0799 - acc: 0.3125
Epoch 5/150
112/112 [==============================] - 0s 541us/step - loss: 1.0691 - acc: 0.3125
Epoch 6/150
112/112 [==============================] - 0s 496us/step - loss: 1.0578 - acc: 0.3125
Epoch 7/150
112/112 [==============================] - 0s 477us/step - loss: 1.0452 - acc: 0.3482
Epoch 8/150
112/112 [==============================] - 0s 483us/step - loss: 1.0306 - acc: 0.3125
Epoch 9/150
112/112 [==============================] - 0s 479us/step - loss: 1.0135 - acc: 0.4196
Epoch 10/150
112/112 [==============================] - 0s 483us/step - loss: 0.9950 - acc: 0.6071
Epoch 11/150
112/112

In [89]:
# Display a summary of the final model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 8)                 40        
_________________________________________________________________
dense_48 (Dense)             (None, 3)                 27        
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________
None


In [90]:
# Evaluate the Keras model on previously unseen data
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))

38/38 [==============================] - 0s 9ms/step

acc: 97.37%

loss: 12.24%


In [91]:
if (notifyStatus): email_notify("Phase 4 Optimize Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 5. Finalize Model

In [92]:
if (notifyStatus): email_notify("Phase 5 Finalize Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [93]:
# Make class predictions with the model
predictions = model.predict_classes(X_original)
print(predictions)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [94]:
if (notifyStatus): email_notify("Phase 5 Finalize Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [95]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:12:33.389036
